In [8]:
#data Preparation

import pandas as pd

#loading the data
train_set = pd.read_csv('semEval dataest/semeval-2017-train.csv')


ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3
